In [ ]:
import os
import sys
from pathlib import Path
import subprocess

# --- Path Setup for Monorepo ---
# Ensure the JennAI root is on sys.path for robust imports of centralized config/core
# This assumes the notebook is in 'JennAI/notebooks/'
jennai_root = Path(os.getcwd()).resolve().parent
if str(jennai_root) not in sys.path:
    sys.path.insert(0, str(jennai_root))

# --- Centralized Logging Setup (Notebook-Specific) ---
# We configure loguru here so logs from this notebook are visible.
# This mimics the setup in main.py.
try:
    from config.loguru_setup import setup_logging
    from config.config import DEBUG_MODE
    setup_logging(debug_mode=DEBUG_MODE)
    from loguru import logger
    logger.info("Loguru initialized for notebook environment.")
except ImportError as e:
    print(f"WARNING: Could not import logging config or Loguru: {e}. Basic prints will be used.")
    logger = None
except Exception as e:
    print(f"ERROR: Failed to set up Loguru: {e}. Basic prints will be used.")
    logger = None

def print_log(level, message):
    if logger:
        getattr(logger, level.lower())(message)
    else:
        print(f"[{level}] {message}")

print_log("INFO", "--- Running Environment Dependency Checks ---")
print_log("INFO", f"Current Working Directory: {os.getcwd()}")
print_log("INFO", f"Python Executable: {sys.executable}")
print_log("INFO", f"sys.path: {sys.path[0:3]}...") 


# --- 1. Python Version ---
print_log("INFO", "\n--- Python Version ---")
try:
    python_version_output = subprocess.run([sys.executable, "--version"], capture_output=True, text=True, check=True)
    print_log("INFO", f"Python Version: {python_version_output.stdout.strip()}")
except Exception as e:
    print_log("ERROR", f"Could not determine Python version: {e}")

# --- 2. Pytest Version ---
print_log("INFO", "\n--- Pytest Version ---")
try:
    pytest_version_output = subprocess.run([sys.executable, "-m", "pytest", "--version"], capture_output=True, text=True, check=True)
    print_log("INFO", f"Pytest Version: {pytest_version_output.stdout.splitlines()[0].strip()}") 
except FileNotFoundError:
    print_log("ERROR", "Pytest executable not found. Is it installed in this environment?")
except Exception as e:
    print_log("ERROR", f"Could not determine Pytest version: {e}")

# --- 3. Google API Key Access ---
print_log("INFO", "\n--- Google API Key Access ---")
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print_log("INFO", f"GOOGLE_API_KEY found (length: {len(google_api_key)} characters).")
else:
    print_log("ERROR", "GOOGLE_API_KEY environment variable NOT found.")
    print_log("INFO", "Please ensure it is set as a Windows System/User Environment Variable and your terminal/VS Code was restarted.")

# --- 4. Loguru Installation (already handled above, but can confirm directly) ---
print_log("INFO", "\n--- Loguru Installation ---")
try:
    import loguru
    print_log("INFO", f"Loguru is installed: Version {loguru.__version__}")
    loguru.logger.info("Test message from Loguru in notebook.")
except ImportError:
    print_log("ERROR", "Loguru is NOT installed in this environment.")
except Exception as e:
    print_log("ERROR", f"Loguru import/usage failed: {e}")

print_log("INFO", "\n--- All checks complete ---")